In [1]:
! pip install snowflake-ml-python -U

     |████████████████████████████████| 1.7MB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 24.6MB 55.7MB/s eta 0:00:01
     |████████████████████████████████| 133kB 111.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.2MB/s  eta 0:00:01
     |████████████████████████████████| 12.2MB 73.5MB/s eta 0:00:01
     |████████████████████████████████| 737kB 96.6MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 101.1MB/s eta 0:00:01
     |████████████████████████████████| 327kB 112.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.8MB/s  eta 0:00:01
     |████████████████████████████████| 200.3MB 25kB/s /s eta 0:00:01
     |████████████████████████████████| 11.1MB 72.6MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 85.9MB/s eta 0:00:01
     |████████████████████████████████| 174kB 99.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 4.5MB/s  eta 0:00:01
     |████████████████████████████████| 194kB 92.9MB/s 

     |████████████████████████████████| 71kB 815kB/s  eta 0:00:01
     |████████████████████████████████| 512kB 93.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 104.3MB/s eta 0:00:01
     |████████████████████████████████| 36.0MB 85.0MB/s eta 0:00:01
     |████████████████████████████████| 256kB 96.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.8MB/s  eta 0:00:01
     |████████████████████████████████| 808kB 83.0MB/s eta 0:00:01
     |████████████████████████████████| 307kB 97.0MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 82.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 3.4MB/s  eta 0:00:01
     |████████████████████████████████| 122kB 95.8MB/s eta 0:00:01
     |████████████████████████████████| 225kB 95.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 4.8MB/s  eta 0:00:01
     |████████████████████████████████| 122kB 95.5MB/s eta 0:00:01
     |████████████████████████████████| 276kB 107.4MB/s eta 0:00

In [1]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.linear_model import SGDClassifier
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

/tmp/pip_packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [3]:
session = get_session()

# Model Building
### Load the final data from snowflake table

In [4]:
table_name = 'predictive_maintenance_final'

In [5]:
sf_df = session.table(table_name).drop('ROW')

In [6]:
sf_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

In [8]:
a = sf_df.to_pandas()

In [11]:
list(a.columns)

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

In [10]:
a.head()

,FAILURE,METRIC5,METRIC6,DAY,DAY_WEEK,SECTOR,OP_PERIOD,DEV_RECONNECTED,MNW1,DIF_M6,DIF_M5,LOG_M2,LOG_M3,LOG_M4,LOG_M7,LOG_M9
0,0,6,407438,1,3,S1F0,1,0,26953834,0,0,2.079442,0.000000,3.970292,0.0,2.079442
1,0,6,403174,1,3,S1F0,1,0,7671335,0,0,0.000000,1.386294,0.000000,0.0,0.000000
2,0,12,237394,1,3,S1F0,1,0,21661996,0,0,0.000000,0.000000,0.000000,0.0,0.000000
3,0,6,410186,1,3,S1F0,1,0,9961753,0,0,0.000000,0.000000,0.000000,0.0,0.000000
4,0,15,313173,1,3,S1F0,1,0,16996310,0,0,0.000000,0.000000,0.000000,0.0,1.386294


In [7]:
CATEGORICAL_COLUMNS = ["SECTOR"]
CATEGORICAL_COLUMNS_OE = ["SECTOR_OE"]
NUMERICAL_COLUMNS = ['METRIC5','METRIC6','DAY','DAY_WEEK','OP_PERIOD','MNW1','DIF_M6','DIF_M5','LOG_M2',
 'LOG_M3','LOG_M4','LOG_M7','LOG_M9']
LABEL_COLUMNS = ["FAILURE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [8]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [9]:
train_df.show(3)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"  |"LOG_M3"            |"LOG_M4"  |"LOG_M7"  |"LOG_M9"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |403174     |1      |3           |S1F0      |1            |0                  |7671335   |0         |0         |0.0       |1.3862943611198906  |0.0       |0.0       |0.0       |
|0          |12         |237394     |1      |3           |S1F0      |1            |0                  |21661996  |0         |0         |0.0       |0.0                 |0.0       |0.0       |0.

Create a pipeline with preprocessing steps and model definition

In [10]:
pipeline = Pipeline(
    steps=[
            (
                "OHE",
                OneHotEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
#             (
#                 "MMS",
#                 MinMaxScaler(
#                 clip=True,
#                 input_cols=NUMERICAL_COLUMNS,
#                 output_cols=NUMERICAL_COLUMNS,
#                 )
#             ),
            (
                "classification",
                SGDClassifier(
                input_cols=NUMERICAL_COLUMNS,
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [11]:
pipeline.fit(train_df)

/tmp/pip_packages/snowflake/snowpark/session.py:1860: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(
/tmp/pip_packages/snowflake/snowpark/session.py:1860: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(
The version of package 'snowflake-snowpark-python' in the local environment is 1.8.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The vers

SnowparkSQLException: (1300) (1304): 01af1b66-0503-cace-0072-f3030bf43e62: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 90, in compute
  File "/tmp/pip_packages/snowflake/ml/modeling/_internal/snowpark_handlers.py", line 90, in fit_wrapper_sproc
  File "/usr/lib/python_udf/47551a7bfd00f4b7db3a8f702d9cb264485e91e604075ad17c0dfcb8e138b05a/lib/python3.8/site-packages/snowflake/snowpark/_internal/telemetry.py", line 76, in wrap
    result = func(*args, **kwargs)
  File "/usr/lib/python_udf/47551a7bfd00f4b7db3a8f702d9cb264485e91e604075ad17c0dfcb8e138b05a/lib/python3.8/site-packages/snowflake/snowpark/dataframe.py", line 492, in to_pandas
    result = self._session._conn.execute(self._plan, to_pandas=True, **kwargs)
  File "/usr/lib/python_udf/47551a7bfd00f4b7db3a8f702d9cb264485e91e604075ad17c0dfcb8e138b05a/lib/python3.8/site-packages/snowflake/snowpark/_internal/server_connection.py", line 354, in execute
    result_set, result_met

In [ ]:
result = pipeline.predict(test_df)